## 初步设计的RAG实现流程如下

### 存储流程

* **原始文件** (text | pdf | excel | ...)
➔ **解析**: `ingestion`
➔ **切片**: `text_splitter`
➔ **向量&存储**: `embedding & store`

### 使用RAG流程

* **用户问题** ➔ **query & 重写**: `multi-query retrieval`
➔ **检索top-k**: `50`
➔ **精排**: `cross encoder`
➔ **LLM生成回答**

---

#### 切片


`RecursiveCharacterTextSplitter` 是 LangChain 提供的文本切分工具：

* **核心思想**：先按大单位切，再按小单位切，保证语义不被切断。
* **作用**：

  * 将长文本拆成 chunk
  * 支持 overlap（防止边界信息丢失）
  * 输出带 metadata 的列表，直接可做 embedding
  

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text = """
C) 上下文（Context）：提供与任务有关的背景信息。这有助于 LLM 理解正在讨论的具体场景，从而确保其响应是相关的。
O) 目标（Objective）：定义你希望 LLM 执行的任务。明晰目标有助于 LLM 将自己响应重点放在完成具体任务上。
S) 风格（Style）：指定你希望 LLM 使用的写作风格。这可能是一位具体名人的写作风格，也可以是某种职业专家（比如商业分析师或 CEO）的风格。这能引导 LLM 使用符合你需求的方式和词语给出响应。
T) 语气（Tone）：设定响应的态度。这能确保 LLM 的响应符合所需的情感或情绪上下文，比如正式、幽默、善解人意等。
A) 受众（Audience）：确定响应的目标受众。针对具体受众（比如领域专家、初学者、孩童）定制 LLM 的响应，确保其在你所需的上下文中是适当的和可被理解的。
R) 响应（Response）：提供响应的格式。这能确保 LLM 输出你的下游任务所需的格式，比如列表、JSON、专业报告等。对于大多数通过程序化方法将 LLM 响应用于下游任务的 LLM 应用而言，理想的输出格式是 JSON。
CO-STAR 的一个实际应用
假设你是一位社交媒体管理者，你需要帮助草拟一篇 Facebook 帖文，其內容是宣传你公司的新产品。
如果不使用 CO-STAR，那么你可能会使用这样的 prompt：
"""
print("\n" in text)
# 创建一个切片器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,   # 每个chunk的大小  实战建议，中文知识库 300~500之间
    chunk_overlap=20,  # chunk之间的重叠  实战建议，中文知识库 50~100之间
    length_function=len,  # 计算长度的函数
    is_separator_regex=False,  # 是否将分隔符视为文本
    separators=["\n\n", "\n", "。", "！", "？", ". ", "! ", "? ", "；", "; ", "，", ", ", " ", ""],
)
chunks = text_splitter.split_text(text)
# 因为文本长度大于chunk_size，所以会切成多个chunk, chunk之间有重叠
print(f"split into {len(chunks)} chunks")
for i, chunk in enumerate(chunks):
    print(f"chunk {i}: {chunk}")

True
split into 2 chunks
chunk 0: C) 上下文（Context）：提供与任务有关的背景信息。这有助于 LLM 理解正在讨论的具体场景，从而确保其响应是相关的。
O) 目标（Objective）：定义你希望 LLM 执行的任务。明晰目标有助于 LLM 将自己响应重点放在完成具体任务上。
S) 风格（Style）：指定你希望 LLM 使用的写作风格。这可能是一位具体名人的写作风格，也可以是某种职业专家（比如商业分析师或 CEO）的风格。这能引导 LLM 使用符合你需求的方式和词语给出响应。
T) 语气（Tone）：设定响应的态度。这能确保 LLM 的响应符合所需的情感或情绪上下文，比如正式、幽默、善解人意等。
A) 受众（Audience）：确定响应的目标受众。针对具体受众（比如领域专家、初学者、孩童）定制 LLM 的响应，确保其在你所需的上下文中是适当的和可被理解的。
R) 响应（Response）：提供响应的格式。这能确保 LLM 输出你的下游任务所需的格式，比如列表、JSON、专业报告等。对于大多数通过程序化方法将 LLM 响应用于下游任务的 LLM 应用而言，理想的输出格式是 JSON。
CO-STAR 的一个实际应用
chunk 1: CO-STAR 的一个实际应用
假设你是一位社交媒体管理者，你需要帮助草拟一篇 Facebook 帖文，其內容是宣传你公司的新产品。
如果不使用 CO-STAR，那么你可能会使用这样的 prompt：


3️⃣ 切分工具的选择原则

| 文档类型            | 推荐切分器                          | 备注         |
| --------------- | ------------------------------ | ---------- |
| PDF / 长手册       | RecursiveCharacterTextSplitter | 保留语义完整     |
| Markdown / Wiki | MarkdownHeaderTextSplitter     | 按章节切       |
| 新闻 / 小说         | SentenceSplitter               | 按句子切       |
| 对话 / 聊天记录       | SentenceSplitter + 自定义轮次       | 保留发言单位     |
| LLM 模型 token 控制 | TokenTextSplitter              | 精确控制 token |



## 问题

### 为什么要切片？

**1. 存储流程中，所选用的向量模型是有长度限制的**

> 几乎所有的向量模型都有严格的输入长度限制，如 `bge-large-zh`：512个token、OpenAI 的 `text-embedding-ada-002` 支持 8192 个 token。

**2. 切片的核心目标是：保证检索的语义精准度**

> 向量化 (Embedding) 的本质是把一段文段压缩成一个几百维的浮点数组。这个数组代表了这段话的“核心主题”，之后我们在检索的时候就依赖于这个核心主题进行相似度计算。
> * **如果切片大小合适**（比如一段话或一页）：这个向量能精准代表这一段的具体细节（比如“2023年Q3苹果公司大中华区营收数据”）。
> * **如果不切片**（整本书算一个向量）：整本书的信息被强行揉捏在一起，就像把满汉全席放进榨汁机打碎一样，最后得到的向量代表的是一个极度模糊的全局概念（比如“一本关于商业的书”）。当你提问具体的细节时，系统根本无法通过相似度匹配找到正确的内容，因为细节的语义完全被长文本里的“噪音”淹没了。
> 
> 

---

### 单从知识的角度理解什么是“完美切片”

最理想的切片方法，本质并不是 **切 (split)** 而是 **重写与提炼 (rewrite & extract)**。

它是把长文档打碎后，重新捏成一个个哪怕完全脱离原文档，也能让路人（或者大模型）一眼看懂的**独立知识块**。

> ps: 目前业界为了达到这种“理想状态”，正在流行一种叫做 Contextual Retrieval（上下文检索） 的前沿方案，也就是让大模型先帮每个切片加上背景信息再存入数据库。


**【案例演示】**

假设你的知识库中有一篇题为《苹果 2023 WWDC 大会纪要》的文档，其中有一段原始文本如下：

> “该公司在当天的发布会上正式推出了首款空间计算设备。它定价3499美元，预计明年发售。CEO蒂姆表示，这不仅是一款新硬件，更是全人类生活方式的革命。为了保证它的产能，团队已经向索尼追加了百万块屏幕的订单。”

#### ❌ 灾难现场：物理切片 (Physical Split)

如果你使用传统的切分器（比如按标点符号或按 50 个字符硬切），你会得到以下切片：

* **Chunk 1:** “该公司在当天的发布会上正式推出了首款空间计算设备。它定价3499美元，预计明年发售。”
* **Chunk 2:** “CEO蒂姆表示，这不仅是一款新硬件，更是全人类生活方式的革命。”
* **Chunk 3:** “为了保证它的产能，团队已经向索尼追加了百万块屏幕的订单。”

**痛点分析：** 假设用户提问：“苹果 Vision Pro 的屏幕是谁供应的？” 系统会去搜索，但 Chunk 3 里只有“它”、“团队”和“索尼”。因为丢失了上下文，向量模型根本不知道“它”就是 Vision Pro，“团队”就是苹果团队。这个包含标准答案的切片，在检索阶段就会被无情淘汰。

#### ✅ 降维打击：重写与提炼 (Rewrite & Extract)

如果采用理想的“信息胶囊”模式（利用大模型在入库前进行指代消解和实体补全），长文档被打碎后，会被重新捏成完全独立的知识块：

* **知识块 1:** “【苹果2023发布会】苹果公司推出了首款空间计算设备（Vision Pro），该设备定价为3499美元，预计于2024年发售。”
* **知识块 2:** “【苹果2023发布会】苹果公司CEO蒂姆·库克表示，Vision Pro的发布不仅是推出新硬件，更是全人类生活方式的革命。”
* **知识块 3:** “【苹果2023发布会】为了保证空间计算设备Vision Pro的产能，苹果团队已经向索尼公司追加了百万块屏幕的订单。”

**优势分析：** 现在，你随便拿出一个知识块扔在马路上，捡到它的路人（或大模型）看一眼就能明白完整的意思。没有让人抓狂的“它”、“这”、“该公司”。当用户再问“苹果 Vision Pro 的屏幕是谁供应的？”时，知识块 3 包含了“苹果”、“Vision Pro”、“产能”、“索尼”、“屏幕”等所有核心实体，向量匹配度会直接爆表，大模型也能基于此给出完美回答。

